In [1]:
!pip install beartype

In [2]:
import seaborn as sns
import pandas as pd
from typing import Optional, List, Literal, Tuple
import numpy as np
from numpy.typing import NDArray
from beartype import beartype

In [3]:
df = sns.load_dataset('titanic')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


## 1. Identify numeric datatype and fill NaN with aggregate statistic values (with an optional groupby)

In [5]:
df['age'].isnull().value_counts()

,count
age,
False,714
True,177


In [6]:
@beartype
def fillna_numeric(df: pd.DataFrame, groupcols: Optional[List[str]] =None,
                   stat_func: Literal['mean', 'median', 'min', 'max', 'std', 'var', 'sum', 'mode'] ='mean') -> pd.DataFrame:
  """
  Fill NaN in numeric columns of `df` with aggregate statistic values (mean by default) with an optional groupby based on columns specified in `groupcols`.
  Inputs:
    df: Dataframe containing all columns
    groupcols: List(columns) or None
    stat_func: 'mean', 'median', 'min', 'max', 'std', 'var', 'sum', 'mode' ('mode' is not recommended when the most frequent value could be NaN)
  Output:
    df_copy: Dataframe with NaN in numeric columns filled
  """
  df_copy = df.copy()
  cols = df.select_dtypes(include=['float64', 'int64']).columns
  try:
    if groupcols:
      df_copy[cols] = df_copy.groupby(groupcols)[cols].transform(lambda x:x.fillna(getattr(x,stat_func)(),inplace=False))
    else:
      df_copy[cols] = df_copy[cols].transform(lambda x:x.fillna(getattr(x,stat_func)(),inplace=False))
  except Exception as e:
    print(e)
  else:
    return df_copy

In [7]:
df['age'].mean(), df['age'][df['sex']=='male'].mean(), df['age'].median(), df['age'][df['sex']=='male'].median()

(np.float64(29.69911764705882), np.float64(30.72664459161148), 28.0, 29.0)

In [8]:
df_1 = fillna_numeric(df, stat_func='median')
display(df['age'][:10],df_1['age'][:10])

,age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0
5,NaN
6,54.0
7,2.0
8,27.0
9,14.0


,age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0
5,28.0
6,54.0
7,2.0
8,27.0
9,14.0


In [9]:
df_2 = fillna_numeric(df,groupcols=['sex'],stat_func='median')
display(df_2['age'][:10])

,age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0
5,29.0
6,54.0
7,2.0
8,27.0
9,14.0


In [10]:
display(df['age'].isnull().value_counts())
display(df_1['age'].isnull().value_counts())
display(df_2['age'].isnull().value_counts())

,count
age,
False,714
True,177


,count
age,
False,891


,count
age,
False,891


## 2. Identify non-numeric data and print their unique values

In [11]:
def unique_nonnumeric_values(df: pd.DataFrame) -> List[Tuple[str,NDArray]]:
  """
  Print unique values of non-numeric columns of `df`.
  Input:
    df: Dataframe containing all columns
  Output:
    List of tuples containing non-numeric column name and unique values
  """
  cols = df.select_dtypes(exclude='number').columns #df.select_dtypes(exclude=['float64', 'int64']).columns
  return([(col,df[col].unique()) for col in cols])

In [12]:
print(unique_nonnumeric_values(df))

[('sex', array(['male', 'female'], dtype=object)), ('embarked', array(['S', 'C', 'Q', nan], dtype=object)), ('class', ['Third', 'First', 'Second']
Categories (3, object): ['First', 'Second', 'Third']), ('who', array(['man', 'woman', 'child'], dtype=object)), ('adult_male', array([ True, False])), ('deck', [NaN, 'C', 'E', 'G', 'D', 'A', 'B', 'F']
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']), ('embark_town', array(['Southampton', 'Cherbourg', 'Queenstown', nan], dtype=object)), ('alive', array(['no', 'yes'], dtype=object)), ('alone', array([False,  True]))]
